In [1]:
!git clone https://github.com/lyuwenyu/RT-DETR.git

Cloning into 'RT-DETR'...
remote: Enumerating objects: 978, done.
remote: Counting objects: 100% (460/460), done.
remote: Compressing objects: 100% (229/229), done.
remote: Total 978 (delta 321), reused 239 (delta 230), pack-reused 518 (from 1)
Receiving objects: 100% (978/978), 602.31 KiB | 18.25 MiB/s, done.
Resolving deltas: 100% (479/479), done.


In [1]:
!pip install -r RT-DETR/rtdetr_paddle/requirements.txt

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
!python -m pip install paddlepaddle-gpu==3.0.0b1 -i https://www.paddlepaddle.org.cn/packages/stable/cu123/
!pip install motmetrics
!export LD_LIBRARY_PATH=/usr/local/cuda/lib64:$LD_LIBRARY_PATH
!export CUDA_VISIBLE_DEVICES=0

#Dataset Preparation

Prepare COCO Dataset : train and val images. \
path/to/coco/ \
  - annotations/  # annotation json files \
  - train2017/    # train images \
  - val2017/      # val images \
Modify config "dataset_dir" on : RT-DETR/rtdetr_paddle/configs/datasets/coco_detection.yml

In [2]:
coco_detection_custom_dataset = """
metric: COCO
num_classes: 8

TrainDataset:
  name: COCODataSet
  image_dir: train/images
  anno_path: train.json
  dataset_dir: /home/user/Documents/detr-training/coco-dataset/
  data_fields: ['image', 'gt_bbox', 'gt_class', 'is_crowd']

EvalDataset:
  name: COCODataSet
  image_dir: val/images
  anno_path: val.json
  dataset_dir: /home/user/Documents/detr-training/coco-dataset/
  allow_empty: true

TestDataset:
  name: ImageFolder
  anno_path: val.json # also support txt (like VOC's label_list.txt)
  dataset_dir: /home/user/Documents/detr-training/coco-dataset/ # if set, anno_path will be 'dataset_dir/anno_path'
"""
coco_detection_custom_config = """
_BASE_: [
  '../datasets/coco_detection_custom.yml',
  '../runtime.yml',
  '_base_/optimizer_6x.yml',
  '_base_/rtdetr_r50vd.yml',
  '_base_/rtdetr_reader.yml',
]

weights: output/rtdetr_r50vd_6x_coco/model_final
find_unused_parameters: True
log_iter: 200

"""

# Membuat dan menulis variabel data ke file
with open("RT-DETR/rtdetr_paddle/configs/datasets/coco_detection_custom.yml", "w") as file:
    file.write(coco_detection_custom_dataset)
with open("RT-DETR/rtdetr_paddle/configs/rtdetr/rtdetr_r50vd_6x_coco_custom.yml", "w") as file:
    file.write(coco_detection_custom_config)

In [17]:
#Training
%cd RT-DETR/rtdetr_paddle/
!python "tools/train.py" -c "configs/rtdetr/rtdetr_r50vd_6x_coco_custom.yml" --eval

[Errno 2] No such file or directory: 'RT-DETR/rtdetr_paddle/'
/home/user/Documents/detr-training/RT-DETR/rtdetr_paddle
======================= Modified FLAGS detected =======================
FLAGS(name='FLAGS_cupti_dir', current_value='/home/user/Documents/detr-training/.venv/lib/python3.10/site-packages/paddle/../nvidia/cuda_cupti/lib', default_value='')
FLAGS(name='FLAGS_cudnn_dir', current_value='/home/user/Documents/detr-training/.venv/lib/python3.10/site-packages/paddle/../nvidia/cudnn/lib', default_value='')
FLAGS(name='FLAGS_cublas_dir', current_value='/home/user/Documents/detr-training/.venv/lib/python3.10/site-packages/paddle/../nvidia/cublas/lib', default_value='')
FLAGS(name='FLAGS_cusolver_dir', current_value='/home/user/Documents/detr-training/.venv/lib/python3.10/site-packages/paddle/../nvidia/cusolver/lib', default_value='')
FLAGS(name='FLAGS_nccl_dir', current_value='/home/user/Documents/detr-training/.venv/lib/python3.10/site-packages/paddle/../nvidia/nccl/lib', defaul

In [4]:
#Evaluation
%cd RT-DETR/rtdetr_paddle/
!python tools/eval.py -c configs/rtdetr/rtdetr_r50vd_6x_coco_custom.yml \
              -o weights=output/rtdetr_r50vd_6x_coco_custom/best_model.pdparams

[Errno 2] No such file or directory: '%cd RT-DETR/rtdetr_paddle/'
/home/user/Documents/detr-training/RT-DETR/rtdetr_paddle
======================= Modified FLAGS detected =======================
FLAGS(name='FLAGS_curand_dir', current_value='/home/user/Documents/detr-training/.venv/lib/python3.10/site-packages/paddle/../nvidia/curand/lib', default_value='')
FLAGS(name='FLAGS_cusparse_dir', current_value='/home/user/Documents/detr-training/.venv/lib/python3.10/site-packages/paddle/../nvidia/cusparse/lib', default_value='')
FLAGS(name='FLAGS_cudnn_dir', current_value='/home/user/Documents/detr-training/.venv/lib/python3.10/site-packages/paddle/../nvidia/cudnn/lib', default_value='')
FLAGS(name='FLAGS_cusolver_dir', current_value='/home/user/Documents/detr-training/.venv/lib/python3.10/site-packages/paddle/../nvidia/cusolver/lib', default_value='')
FLAGS(name='FLAGS_cupti_dir', current_value='/home/user/Documents/detr-training/.venv/lib/python3.10/site-packages/paddle/../nvidia/cuda_cupti/

In [3]:
#Inference
# !pip install sahi
%cd RT-DETR/rtdetr_paddle/
!python tools/infer.py -c configs/rtdetr/rtdetr_r50vd_6x_coco_custom.yml \
              -o weights=output/rtdetr_r50vd_6x_coco_custom/best_model.pdparams \
              --infer_dir="/home/user/Documents/detr-training/test-inference/" \
            #   --slice_infer \
              --slice_size 1920

/home/user/Documents/detr-training/RT-DETR/rtdetr_paddle


/home/user/Documents/detr-training/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


W1028 02:04:16.035501 211947 gpu_resources.cc:119] Please NOTE: device: 0, GPU Compute Capability: 8.9, Driver API Version: 12.4, Runtime API Version: 12.3
W1028 02:04:16.036798 211947 gpu_resources.cc:164] device: 0, cuDNN Version: 9.0.
[10/28 02:04:17] ppdet.utils.checkpoint INFO: Finish loading model weights: output/rtdetr_r50vd_6x_coco_custom/best_model.pdparams
[10/28 02:04:17] train INFO: Found 15 inference images in total.
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
100%|███████████████████████████████████████████| 15/15 [00:01<00:00,  9.77it/s]
[10/28 02:04:19] ppdet.engine INFO: Detection bbox results save in output/4b2445b1-7333-4c85-968a-00788fe3a693.png
[10/28 02:04:19] ppdet.engine INFO: Detection bbox results save in output/4ac7b4e0-a7f8-468a-b2b3-773ba6c9e9cc.png
[10/28 02:04:19] ppdet.engine INFO: Detection bbox results save in output/583a564f-0cff-4c9a-b27a-af1dda6aa0be.png
[10/28 02:04:19] ppdet.engine INFO: Detection bbox result

In [4]:
#Inference
%cd RT-DETR/rtdetr_paddle/
!python tools/infer.py -c configs/rtdetr/rtdetr_r50vd_6x_coco_custom.yml \
              -o weights=output/rtdetr_r50vd_6x_coco_custom/best_model.pdparams \
              --infer_img="/home/user/Pictures/Screenshot_20241018_084437.png" \
            #   --slice_infer \
              --slice_size 1920

[Errno 2] No such file or directory: 'RT-DETR/rtdetr_paddle/'
/home/user/Documents/detr-training/RT-DETR/rtdetr_paddle
W1028 02:04:27.491632 212042 gpu_resources.cc:119] Please NOTE: device: 0, GPU Compute Capability: 8.9, Driver API Version: 12.4, Runtime API Version: 12.3
W1028 02:04:27.492712 212042 gpu_resources.cc:164] device: 0, cuDNN Version: 9.0.
[10/28 02:04:28] ppdet.utils.checkpoint INFO: Finish loading model weights: output/rtdetr_r50vd_6x_coco_custom/best_model.pdparams
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  1.82it/s]
[10/28 02:04:29] ppdet.engine INFO: Detection bbox results save in output/Screenshot_20241018_084437.png


In [11]:

%cd RT-DETR/rtdetr_paddle/
inference = '''

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import sys
import cv2
import numpy as np
import tempfile

# add python path of PaddleDetection to sys.path
parent_path = os.path.abspath(os.path.join(__file__, *(['..'] * 2)))
sys.path.insert(0, parent_path)

# ignore warning log
import warnings
warnings.filterwarnings('ignore')
import glob
import ast

import paddle
from ppdet.core.workspace import load_config, merge_config
from ppdet.engine import Trainer
from ppdet.utils.check import check_gpu, check_npu, check_xpu, check_mlu, check_version, check_config
from ppdet.utils.cli import ArgsParser, merge_args

from ppdet.utils.logger import setup_logger
logger = setup_logger('train')


def parse_args():
    parser = ArgsParser()
    parser.add_argument(
        "--infer_dir",
        type=str,
        default=None,
        help="Directory for images to perform inference on.")
    parser.add_argument(
        "--infer_img",
        type=str,
        default=None,
        help="Image path, has higher priority over --infer_dir")
    parser.add_argument(
        "--output_dir",
        type=str,
        default="output",
        help="Directory for storing the output visualization files.")
    parser.add_argument(
        "--draw_threshold",
        type=float,
        default=0.5,
        help="Threshold to reserve the result for visualization.")
    parser.add_argument(
        "--slim_config",
        default=None,
        type=str,
        help="Configuration file of slim method.")
    parser.add_argument(
        "--use_vdl",
        type=bool,
        default=False,
        help="Whether to record the data to VisualDL.")
    parser.add_argument(
        '--vdl_log_dir',
        type=str,
        default="vdl_log_dir/image",
        help='VisualDL logging directory for image.')
    parser.add_argument(
        "--save_results",
        type=bool,
        default=False,
        help="Whether to save inference results to output_dir.")
    parser.add_argument(
        "--slice_infer",
        action='store_true',
        help="Whether to slice the image and merge the inference results for small object detection."
    )
    parser.add_argument(
        '--slice_size',
        nargs='+',
        type=int,
        default=[640, 640],
        help="Height of the sliced image.")
    parser.add_argument(
        "--overlap_ratio",
        nargs='+',
        type=float,
        default=[0.25, 0.25],
        help="Overlap height ratio of the sliced image.")
    parser.add_argument(
        "--combine_method",
        type=str,
        default='nms',
        help="Combine method of the sliced images' detection results, choose in ['nms', 'nmm', 'concat']."
    )
    parser.add_argument(
        "--match_threshold",
        type=float,
        default=0.6,
        help="Combine method matching threshold.")
    parser.add_argument(
        "--match_metric",
        type=str,
        default='ios',
        help="Combine method matching metric, choose in ['iou', 'ios'].")
    parser.add_argument(
        "--visualize",
        type=ast.literal_eval,
        default=True,
        help="Whether to save visualize results to output_dir.")
    args = parser.parse_args()
    return args


def get_test_images(infer_dir, infer_img):
    """
    Get image path list in TEST mode
    """
    assert infer_img is not None or infer_dir is not None,         "--infer_img or --infer_dir should be set"
    assert infer_img is None or os.path.isfile(infer_img),             "{} is not a file".format(infer_img)
    assert infer_dir is None or os.path.isdir(infer_dir),             "{} is not a directory".format(infer_dir)

    # infer_img has a higher priority
    if infer_img and os.path.isfile(infer_img):
        return [infer_img]

    images = set()
    infer_dir = os.path.abspath(infer_dir)
    assert os.path.isdir(infer_dir),         "infer_dir {} is not a directory".format(infer_dir)
    exts = ['jpg', 'jpeg', 'png', 'bmp']
    exts += [ext.upper() for ext in exts]
    for ext in exts:
        images.update(glob.glob('{}/*.{}'.format(infer_dir, ext)))
    images = list(images)

    assert len(images) > 0, "no image found in {}".format(infer_dir)
    logger.info("Found {} inference images in total.".format(len(images)))

    return images

def preprocess_image(input_path, invert_binary=False):
    logger.info(f"Processing image: {input_path}")
    
    # Check if file exists
    if not os.path.exists(input_path):
        logger.error(f"File not found: {input_path}")
        return None

    # Load image in grayscale
    img = cv2.imread(input_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        logger.error(f"Failed to load image: {input_path}")
        return None

    # Apply adaptive thresholding
    threshold_type = cv2.THRESH_BINARY_INV if invert_binary else cv2.THRESH_BINARY
    binary = cv2.adaptiveThreshold(
        img,
        maxValue=255,
        adaptiveMethod=cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        thresholdType=threshold_type,
        blockSize=11,
        C=2
    )
    
    # Add border
    h, w = binary.shape
    bordered = np.zeros((h + 2, w + 2), dtype=np.uint8)
    bordered[1:-1, 1:-1] = binary
    
    # Convert back to BGR for compatibility with the model
    bordered_bgr = cv2.cvtColor(bordered, cv2.COLOR_GRAY2BGR)
    
    # Save preprocessed image to a temporary file
    with tempfile.NamedTemporaryFile(delete=False, suffix='.jpg') as tmp_file:
        cv2.imwrite(tmp_file.name, bordered_bgr)
        logger.info(f"Preprocessed image saved to: {tmp_file.name}")
        return tmp_file.name

def run(FLAGS, cfg):
    trainer = Trainer(cfg, mode='test')
    trainer.load_weights(cfg.weights)
    # get inference images
    image_paths = get_test_images(FLAGS.infer_dir, FLAGS.infer_img)
    logger.info(f"Found {len(image_paths)} images")

    # Preprocess images
    preprocessed_image_paths = []
    for img_path in image_paths:
        processed_path = preprocess_image(img_path)
        if processed_path:
            preprocessed_image_paths.append(processed_path)

    if not preprocessed_image_paths:
        logger.error("No valid preprocessed images found")
        return

    logger.info(f"Preprocessed {len(preprocessed_image_paths)} images")

    # inference
    if FLAGS.slice_infer:
        trainer.slice_predict(
            preprocessed_image_paths,
            slice_size=FLAGS.slice_size,
            overlap_ratio=FLAGS.overlap_ratio,
            combine_method=FLAGS.combine_method,
            match_threshold=FLAGS.match_threshold,
            match_metric=FLAGS.match_metric,
            draw_threshold=FLAGS.draw_threshold,
            output_dir=FLAGS.output_dir,
            save_results=FLAGS.save_results,
            visualize=FLAGS.visualize)
    else:
        trainer.predict(
            preprocessed_image_paths,
            draw_threshold=FLAGS.draw_threshold,
            output_dir=FLAGS.output_dir,
            save_results=FLAGS.save_results,
            visualize=FLAGS.visualize)

    # Clean up temporary files
    for path in preprocessed_image_paths:
        os.remove(path)

def main():
    FLAGS = parse_args()
    cfg = load_config(FLAGS.config)
    merge_args(cfg, FLAGS)
    merge_config(FLAGS.opt)

    # disable npu in config by default
    if 'use_npu' not in cfg:
        cfg.use_npu = False

    # disable xpu in config by default
    if 'use_xpu' not in cfg:
        cfg.use_xpu = False

    if 'use_gpu' not in cfg:
        cfg.use_gpu = False

    # disable mlu in config by default
    if 'use_mlu' not in cfg:
        cfg.use_mlu = False

    if cfg.use_gpu:
        place = paddle.set_device('gpu')
    elif cfg.use_npu:
        place = paddle.set_device('npu')
    elif cfg.use_xpu:
        place = paddle.set_device('xpu')
    elif cfg.use_mlu:
        place = paddle.set_device('mlu')
    else:
        place = paddle.set_device('cpu')

    check_config(cfg)
    check_gpu(cfg.use_gpu)
    check_npu(cfg.use_npu)
    check_xpu(cfg.use_xpu)
    check_mlu(cfg.use_mlu)
    check_version()

    run(FLAGS, cfg)


if __name__ == '__main__':
    main()
'''

with open("tools/inference_custom.py", "w") as file:
    file.write(inference)

[Errno 2] No such file or directory: 'RT-DETR/rtdetr_paddle/'
/home/user/Documents/detr-training/RT-DETR/rtdetr_paddle


In [6]:
#Inference
%cd RT-DETR/rtdetr_paddle/
!python tools/inference_custom.py -c configs/rtdetr/rtdetr_r50vd_6x_coco_custom.yml \
              -o weights=output/rtdetr_r50vd_6x_coco_custom/best_model.pdparams \
              --infer_img="/home/user/Pictures/Screenshot_20241029_001310.png" \
            #   --slice_infer \
              --slice_size 1920 \
              --draw_threshold 0.1

[Errno 2] No such file or directory: 'RT-DETR/rtdetr_paddle/'
/home/user/Documents/detr-training/RT-DETR/rtdetr_paddle
W1029 00:21:23.139829  9388 gpu_resources.cc:119] Please NOTE: device: 0, GPU Compute Capability: 8.9, Driver API Version: 12.4, Runtime API Version: 12.3
W1029 00:21:23.141101  9388 gpu_resources.cc:164] device: 0, cuDNN Version: 9.0.
[10/29 00:21:24] ppdet.utils.checkpoint INFO: Finish loading model weights: output/rtdetr_r50vd_6x_coco_custom/best_model.pdparams
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  1.94it/s]
[10/29 00:21:25] ppdet.engine INFO: Detection bbox results save in output/Screenshot_20241029_001310.png
